In [3]:
# ===============================================================
# CELL 5 — Full Observation Simulation (Module 11)
# ===============================================================

import numpy as np
from astropy.io import fits
from scipy.ndimage import gaussian_filter
import json
import os

print("Starting Full Observation Simulation...\n")

# ---------------------------------------------------------------
# USER-CONTROLLED PARAMETERS
# ---------------------------------------------------------------
NUM_EXPOSURES = 8               # number of frames to simulate
SEEING_MEAN = 2.0               # arcsec FWHM
SEEING_VARIATION = 0.5          # arcsec random variation
TELESCOPE_DRIFT_PIX = 1.5       # max pixel drift per frame
BACKGROUND_MEAN = 50
NOISE_STD = 5
PLATE_SCALE = 0.40   # arcsec/pixel

PIX_SCALE = PLATE_SCALE         # arcsec/pixel (from earlier module)
FWHM_TO_SIG = 1 / (2*np.sqrt(2*np.log(2)))

# ---------------------------------------------------------------
# FUNCTION — generate a single exposure
# ---------------------------------------------------------------
def generate_exposure(base_img, frame_id):
    # 1. Drift (shift stars)
    dy = np.random.uniform(-TELESCOPE_DRIFT_PIX, TELESCOPE_DRIFT_PIX)
    dx = np.random.uniform(-TELESCOPE_DRIFT_PIX, TELESCOPE_DRIFT_PIX)

    shifted = np.roll(base_img, int(dy), axis=0)
    shifted = np.roll(shifted, int(dx), axis=1)

    # 2. Seeing variation → PSF blur change
    seeing_now = np.random.normal(SEEING_MEAN, SEEING_VARIATION)
    sigma_pix = (seeing_now / PIX_SCALE) * FWHM_TO_SIG

    blurred = gaussian_filter(shifted, sigma_pix)

    # 3. Sky background
    background = np.random.normal(BACKGROUND_MEAN, 3, blurred.shape)

    # 4. Sensor read noise
    noise = np.random.normal(0, NOISE_STD, blurred.shape)

    final = blurred + background + noise

    return final, {
        "frame_id": frame_id,
        "drift_dx_pix": float(dx),
        "drift_dy_pix": float(dy),
        "seeing_fwhm_arcsec": float(seeing_now)
    }

# ---------------------------------------------------------------
# GENERATE ALL EXPOSURES
# ---------------------------------------------------------------
exposures = []
log = {"num_exposures": NUM_EXPOSURES, "frames": []}

for i in range(NUM_EXPOSURES):
    frame, info = generate_exposure(img, i + 1)
    exposures.append(frame)
    log["frames"].append(info)
    
    fits.writeto(f"exposure_{i+1}.fits", frame, overwrite=True)

print("Generated", NUM_EXPOSURES, "exposures.")

# ---------------------------------------------------------------
# STACK IMAGES
# ---------------------------------------------------------------
stack_median = np.median(exposures, axis=0)
stack_mean   = np.mean(exposures, axis=0)

fits.writeto("stack_median.fits", stack_median, overwrite=True)
fits.writeto("stack_mean.fits",   stack_mean,   overwrite=True)

print("Saved stack_median.fits and stack_mean.fits")

# ---------------------------------------------------------------
# SAVE OBSERVATION LOG
# ---------------------------------------------------------------
with open("observation_log.json", "w") as f:
    json.dump(log, f, indent=4)

print("Saved observation_log.json")


Starting Full Observation Simulation...

Generated 8 exposures.
Saved stack_median.fits and stack_mean.fits
Saved observation_log.json
